<a href="https://colab.research.google.com/github/laurabustamante/foraging_tutorial/blob/main/foraging_tutorial_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Welcome to the tutorial!**
**Foraging theory: models and applications to effort-based decision-making**
<br>
This tutorial was written by Laura Ana Bustamante in 2024 from Washington University in St. Louis. Much of the code was adapted from scripts provided to me by Nathaniel Daw, Sean Devine, and Sam Zorowitz, many thanks to them.
<br>
In this tutorial you will learn about how to run simulations to get the optimal policy in a patch foraging environment. Then you will estimate individual differences in willingness to exert cognitive and physical effort by fitting a hierarchical Bayesian MVT model. We will also cover how to utilize hierarchical models to estimate correlations between inter-parameter individual differences. We will use open source Effort Foraging Task data from Bustamante and colleagues (2023).
<br>
See the [paper](https://www.pnas.org/doi/10.1073/pnas.2221510120) and [OSF](https://osf.io/a4r2e/) repository for complete information.

In [1]:
# Define the optimal_threshold function
optimal_threshold <- function(d, h, mu, sigma, a, b, time) {

  # Initialize variables
  thresh_old <- 4
  tol <- 1e-3
  max_iter <- 200
  diff <- 1e3
  iter_opt <- 0

  # Iterate until convergence or maximum iterations reached
  while (diff > tol && iter_opt < max_iter) {
    iter_opt <- iter_opt + 1
    # Call evaluate_threshold function to get results
    results <- evaluate_threshold(thresh_old, max_iter, d, h, mu, sigma, a, b, time)
    # Update threshold based on average reward rate
    thresh_new <- results$rho_mean * h
    diff <- abs(thresh_new - thresh_old)
    thresh_old <- thresh_new
    # Display iteration information
    cat("iteration", iter_opt, "...")
    cat("... threshold value =", thresh_old, "...")
    cat("... average rho =", results$rho_mean, "\n")
  }

  # Return results as a list
  list(thresh = thresh_new,
       rho = results$rho_mean,
       n_harvests_mean = results$n_harvest_mean,
       n_harvests_sd = results$n_harvest_sd,
       n_exits_mean = results$n_exit_mean,
       n_exits_sd = results$n_exit_sd)
}

# Define the evaluate_threshold function
evaluate_threshold <- function(thresh, iter_eval, d, h, mu, sigma, a, b, time) {

  # Initialize variables
  r_max <- mu + 5
  E_kappa <- a / (a + b)
  exit_harvest_counter <- c()
  rho <- numeric(iter_eval)
  num_exit <- numeric(iter_eval)

  # Iterate for the specified number of evaluations
  for (i in 1:iter_eval) {
    t_counter <- 0
    r_counter <- 0
    harvest_counter <- 0
    exit_harvest_counter_i <- c()

    # Simulate foraging process
    while ((t_counter + h) <= time) {
      if (harvest_counter == 0) {
        # Get initial reward from a normal distribution
        r <- rnorm(1, mean = mu, sd = sigma)
        if (r > r_max) {
          r <- r_max
        }
        t_counter <- t_counter + h
        r_counter <- r_counter + r
        harvest_counter <- harvest_counter + 1
      } else {
        # Calculate expected reward based on decay rate
        Er <- E_kappa * r
        if (Er < thresh) {
          # If expected reward is below threshold, exit
          t_counter <- t_counter + d
          exit_harvest_counter <- c(exit_harvest_counter, harvest_counter)
          exit_harvest_counter_i <- c(exit_harvest_counter_i, harvest_counter)
          harvest_counter <- 0
        } else {
          # If expected reward is above threshold, continue harvesting
          kappa <- rbeta(1, shape1 = a, shape2 = b)
          r <- kappa * r
          r_counter <- r_counter + r
          t_counter <- t_counter + h
          harvest_counter <- harvest_counter + 1
        }
      }
    }
    # Calculate average reward rate and number of exits
    rho[i] <- r_counter / time
    num_exit[i] <- length(exit_harvest_counter_i)
  }

  # Return results as a list
  list(rho_mean = mean(rho),
       n_harvest_mean = mean(exit_harvest_counter),
       n_harvest_sd = sd(exit_harvest_counter),
       n_exit_mean = mean(num_exit),
       n_exit_sd = sd(num_exit))
}

Now we are going to find the best threshold for the environment use in Experiment 1 of Bustamante et al. 2023.

In [2]:
# call the optimal_threshold function with inputs: (d, h, mu, sigma, a, b, time)
# d = travel time, h = harvest time; start reward, normal distribution: mu, sigma; decay, beta distribution: a, b, time = total orchard time
result <- optimal_threshold(8.33, 2, 15, 1, 14.90873, 2.033008, 480)

# Display the results
print(paste("best exit threshold:",
            round(result$thresh,5)))
print(paste("reward rate achieved:",
            round(result$rho,5)))
print(paste("mean number of harvests before exit:",
            round(result$n_harvests_mean,5)))
print(paste("sd number of harvests before exit:",
            round(result$n_harvests_sd,5)))
print(paste("mean number of exits within simulation time:",
            round(result$n_exits_mean,5)))
print(paste("sd number of exits within simulation time:",
            round(result$n_exits_sd,5)))

iteration 1 ...... threshold value = 6.410235 ...... average rho = 3.205117 
iteration 2 ...... threshold value = 6.770819 ...... average rho = 3.385409 
iteration 3 ...... threshold value = 6.78942 ...... average rho = 3.39471 
iteration 4 ...... threshold value = 6.787234 ...... average rho = 3.393617 
iteration 5 ...... threshold value = 6.769978 ...... average rho = 3.384989 
iteration 6 ...... threshold value = 6.799668 ...... average rho = 3.399834 
iteration 7 ...... threshold value = 6.769131 ...... average rho = 3.384565 
iteration 8 ...... threshold value = 6.802007 ...... average rho = 3.401004 
iteration 9 ...... threshold value = 6.791196 ...... average rho = 3.395598 
iteration 10 ...... threshold value = 6.786095 ...... average rho = 3.393048 
iteration 11 ...... threshold value = 6.7758 ...... average rho = 3.3879 
iteration 12 ...... threshold value = 6.77458 ...... average rho = 3.38729 
iteration 13 ...... threshold value = 6.802476 ...... average rho = 3.401238 
ite

**Question 1:** predict how the exit threshold would change if you increase the average starting reward from 15 to 20 apples? Write code below to find out:

In [ ]:
# Question 1, find the best exit threshold when the starting reward (mu) is set to 20 apples

**Question 2:** which parameters of the environment could you change to decrease the exit threshold? confirm your predictions with code


In [ ]:
# Question 1, starting from the environment used in Experiment 1
# reference:
# result <- optimal_threshold(8.33, 2, 15, 1, 14.90873, 2.033008, 480)
# make as many changes as you can think of to decrease the best exit threshold

In [3]:
urlfile="https://github.com/laurabustamante/foraging_tutorial"

choiceData_experiment_1<-read_csv(url(urlfile))

[1] "/content"

Do human participants apply the optimal threshold? We will look at behavior in Experiment 1 from the low effort conditions (cognitive and physical)

In [ ]:
setwd("~/data/experiment_1")
choiceData_experiment_1 <- read_csv("choiceData_experiment_1.csv")
setwd('~/hierarchal_bayesian_mvt_model/')
cost_travel_stan_fit_experiment_1 <- fread("cmdstan_samples_allsubs_experiment_1.tsv.gz") %>%
  mutate(rownum = row_number(),
         Chain = case_when(rownum > 0 & rownum <= 1000 ~ 1,
                           rownum > 1000 & rownum <= 2000 ~ 2,
                           rownum > 2000 & rownum <= 3000 ~ 3,
                           rownum > 3000 & rownum <= 4000 ~ 4))
cost_travel_fit_vars <- c("inv_temp",
                          "cost_matching",
                          "cost_mismatching",
                          "cost_small",
                          "cost_large")
cost_travel_fit_vars_titles <- c("Inverse Tempurature",
                                "Cognitive Low Effort Travel Cost",
                                "Cognitive Effort Cost",
                                "Physical Low Effort Travel Cost",
                                "Physical Effort Cost")
cost_travel_participant_experiment_1 <- read_csv("cost_travel_participant_experiment_1.csv") %>%
   mutate(param_name_str = case_when(param_name == "cost_large" ~ "Physical effort cost",
                                    param_name == "cost_small" ~ "Physical low effort cost",
                                    param_name == "cost_matching" ~ "Cognitive low effort cost",
                                    param_name == "cost_mismatching" ~ "Cognitive effort cost",
                                    param_name == "inv_temp" ~ "Inverse tempurature"))

In [ ]:
### Figure S2: Individual differences in overall threshold.
lmer_threshold_allcond_subj_experiment_1 <- lmer('expectedReward ~ 1 + (1 | subject_id)',
                                               data = choiceData_experiment_1 %>%
                                                 filter(decision == "exit",
                                                        effort_level == "low"))
exit_threshold_subj_experiment_1 <- ranef(lmer_threshold_allcond_subj_experiment_1)$subject_id %>%
  mutate(subject_id = row.names(.)) %>%
  rename(threshold_subj = "(Intercept)") %>%
  mutate(Threshold = threshold_subj + fixef(lmer_threshold_allcond_subj_experiment_1))
threshold_subj_max <- max(max(exit_threshold_subj_experiment_1$Threshold))
figure_S2 <- ggplot(data = exit_threshold_subj_experiment_1) +
  geom_histogram(aes(Threshold),
                 binwidth = 0.5) +
  labs(y = "Number of Participants",
       x = "Overall exit threshold (apples)") +
  scale_y_continuous(breaks = scales::breaks_pretty(n = 3)) +
  scale_x_continuous(limits = c(0,
                                threshold_subj_max),
                     breaks = scales::breaks_pretty()) +
  theme(text=element_text(size = 14,
                          family = "Times New Roman"),
        axis.text = element_text(size = 14),
        panel.background = element_blank(),
        panel.grid.major = element_line(size = 0.5,
                                        linetype = 'solid',
                                        colour = "grey60")) +
  geom_vline(aes(xintercept = 6.78),
             linetype = "dashed")
print(figure_S2)

In [ ]:
### Figure 3: Change in exit thresholds by effort condition.
# participant
change_exit_threshold_participant_experiment_1 <- choiceData_experiment_1 %>%
  filter(decision == "exit") %>%
  group_by(subject_id, travel_task, effort_level) %>%
  summarise(exit_threshold = mean(expectedReward, na.rm = TRUE)) %>%
  pivot_wider(names_from = c("travel_task",
                             "effort_level"),
              values_from = c("exit_threshold")) %>%
  mutate(change_exit_msit = msit_high - msit_low,
         change_exit_bp = bp_high - bp_low) %>%
  ungroup() %>%
  select(subject_id,
         change_exit_msit,
         change_exit_bp) %>%
  pivot_longer(-subject_id) %>%
  mutate(effort_type = case_when(name == "change_exit_msit" ~ "Cognitive (MSIT)",
                                 name == "change_exit_bp" ~ "Physical"),
         effort_type = factor(effort_type,
                              levels = c("Cognitive (MSIT)",
                                         "Physical")))
change_exit_threshold_min <- min(change_exit_threshold_participant_experiment_1$value)
change_exit_threshold_max <- max(change_exit_threshold_participant_experiment_1$value)

# group
nsubj_experiment_1 <- length(unique(choiceData_experiment_1$subject_id))
change_exit_threshold_group_experiment_1 <- change_exit_threshold_participant_experiment_1 %>%
  group_by(effort_type) %>%
  summarise(change_threshold_group_mean = mean(value),
            change_threshold_group_sem = sd(value)/sqrt(nsubj_experiment_1))
change_threshold_group_min <- min(change_exit_threshold_group_experiment_1$change_threshold_group_mean -
                         change_exit_threshold_group_experiment_1$change_threshold_group_sem)
# plot
p_change_exit_threshold_group_experiment_1 <- ggplot(data = change_exit_threshold_group_experiment_1,
       aes(y = change_threshold_group_mean, x = effort_type)) +
  geom_bar(stat = "identity", position = position_dodge(width=0.9)) +
  labs(y = "Change exit threshold (high - low effort, apples)",
       tag = "A") +
    geom_errorbar(aes(ymax = change_threshold_group_mean + change_threshold_group_sem,
                    ymin= change_threshold_group_mean - change_threshold_group_sem),
                  width = 0,
                  size = 1,
                  position = position_dodge(width=0.9),
                  color = "grey20") +
  geom_hline(yintercept = 0,
             color = "black") +
  scale_y_continuous(limits = c(change_threshold_group_min, 0),
                     breaks = scales::breaks_pretty()) +
  theme(text = element_text(size = 14,
                            family = "Times New Roman"),
        axis.text = element_text(size = 14),
        axis.title.x = element_blank(),
        panel.background = element_blank(),
        panel.grid.major = element_line(size = 0.5,
                                          linetype = 'solid',
                                colour = "grey60"),
        panel.grid.major.x = element_blank())

p_change_exit_threshold_participant_experiment_1 <- ggplot(data = change_exit_threshold_participant_experiment_1) +
  geom_histogram(aes(value),
                 binwidth = 0.25) +
  facet_wrap(~ effort_type,
             ncol = 1) +
  labs(y = "Number of participants",
       x = "Change exit threshold (high - low effort, apples)",
       tag = "B") +
  scale_y_continuous(breaks = scales::breaks_pretty()) +
  scale_x_continuous(limits = c(change_exit_threshold_min,
                                change_exit_threshold_max),
                     breaks = scales::breaks_pretty()) +
  theme(text=element_text(size = 14,
                          family = "Times New Roman"),
        axis.text = element_text(size = 14),
        panel.background = element_blank(),
        panel.grid.major = element_line(size = 0.5,
                                          linetype = 'solid',
                                colour = "grey60")) +
  geom_vline(xintercept = 0)

figure_3 <- ggdraw() +
  draw_plot(p_change_exit_threshold_group_experiment_1,
            x = 0,
            y = 0,
            width = 0.4,
            height = 1) +
  draw_plot(p_change_exit_threshold_participant_experiment_1,
          x = 0.4,
          y = 0,
          width = 0.6,
          height = 1)
print(figure_3)

In [ ]:
### Figure 4: Correlation between individual differences in cognitive and physical effort costs.
# histogram of individual differences in effort cost
cost_min <- min(filter(cost_travel_participant_experiment_1,
                      param_name == "cost_mismatching" |
                      param_name == "cost_large")$lower)
cost_max <- max(filter(cost_travel_participant_experiment_1,
                      param_name == "cost_mismatching" |
                      param_name == "cost_large")$upper)
p_cost_travel_histogram_experiment_1 <- ggplot(data = cost_travel_participant_experiment_1 %>%
         filter(param_name == "cost_mismatching" |
                param_name == "cost_large")) +
  geom_histogram(aes(mean),
                 binwidth = 2) +
  facet_wrap(~ param_name_str, ncol=1) +
  labs(y = "Number of participants",
       x = " ",
       tag = "A") +
  geom_vline(aes(xintercept = 0),
             color = "black") +
  theme(text = element_text(size = 14,
                            family = "Times New Roman"),
        axis.text = element_text(size = 14),
        strip.background = element_rect(fill="white"),
        strip.text = element_text(size = 14,
                                  colour="black"),
        panel.background = element_blank(),
        panel.grid.major = element_line(size = 0.5,
                                        linetype = 'solid',
                                        colour = "grey60")) +
  scale_y_continuous(breaks=scales::breaks_pretty()) +
  scale_x_continuous(breaks=scales::breaks_pretty(),
                     limits = c(cost_min,
                                cost_max))
# scatter plot cognitive versus physical effort cost with HDI intervals
cost_travel_param_participant_experiment_1 <- cost_travel_participant_experiment_1 %>%
         mutate(HDIInterval_width = upper - lower) %>%
         ungroup() %>%
         select(-param_name_str) %>%
         pivot_wider(names_from = param_name,
                     values_from = c(mean, lower, upper, HDIInterval_width))
cost_max <- max(max(cost_travel_param_participant_experiment_1$upper_cost_large),
                max(cost_travel_param_participant_experiment_1$upper_cost_mismatching))
cost_min <- min(min(cost_travel_param_participant_experiment_1$lower_cost_large),
                min(cost_travel_param_participant_experiment_1$lower_cost_mismatching))
p_cost_travel_scatter_experiment_1 <- ggplot(data = cost_travel_param_participant_experiment_1,
       aes(x = mean_cost_mismatching,
           y = mean_cost_large)) +
  labs(x = "Cognitive Effort Cost",
       y = "Physical Effort Cost",
       tag = "B") +
  geom_vline(aes(xintercept = 0),
             color = "black") +
  geom_hline(aes(yintercept = 0),
           color = "black") +
  geom_errorbar(aes(xmin = lower_cost_mismatching,
                    xmax = upper_cost_mismatching),
                alpha = 0.1) +
  geom_errorbar(aes(ymin = lower_cost_large,
                    ymax = upper_cost_large),
                alpha = 0.1) +
  geom_point(color = "black",
             fill = "white",
             alpha = 0.75,
             pch=21) +
  theme(text = element_text(size = 14,
                            family = "Times New Roman"),
        axis.text = element_text(size = 14),
        panel.background = element_blank(),
        panel.grid.major = element_line(size = 0.5,
                                        linetype = 'solid',
                                        colour = "grey60")) +
    coord_fixed(xlim = c(cost_min, cost_max),
                ylim = c(cost_min, cost_max)) +
  scale_y_continuous(breaks=scales::breaks_pretty()) +
  scale_x_continuous(breaks=scales::breaks_pretty())

p_cog_phys_correlation_posterior_experiment_1 <- ggplot(data = cost_travel_stan_fit_experiment_1 %>%
                                                          select(`r[3,5]`),
       aes(y = 0,
           x = `r[3,5]`,
           fill = stat(quantile))) +
  ggridges::geom_density_ridges_gradient(quantile_lines = TRUE,
                               quantile_fun = hdi,
                               vline_linetype = 2) +
  geom_vline(aes(xintercept = mean(cost_travel_stan_fit_experiment_1$`r[3,5]`)),
           color = "grey20") +
  labs(y = "Posterior density",
       x = "Correlation of cognitive and \n physical effort costs",
       tag = "C") +
  theme(text = element_text(family = "Times New Roman",
                            size = 14),
        axis.text = element_blank(),
        axis.ticks.y = element_blank(),
        axis.text.x = element_text(family = "Times New Roman",
                            size = 12),
        panel.background = element_blank(),
        panel.grid.major = element_line(size = 0.5,
                                        linetype = 'solid',
                                        colour = "grey60")) +
  scale_x_continuous(breaks=scales::pretty_breaks()) +
  scale_fill_manual(values = c("transparent",
                               "grey50",
                               "transparent"),
                    guide = "none")

figure_4 <- ggdraw() +
  draw_plot(p_cost_travel_histogram_experiment_1,
            x = 0,
            y = 0.35,
            width = 0.5,
            height = 0.65) +
  draw_plot(p_cost_travel_scatter_experiment_1,
            x = 0.5,
            y = 0.35,
            width = 0.5,
            height = 0.65) +
  draw_plot(p_cog_phys_correlation_posterior_experiment_1,
            x = 0.2,
            y = 0,
            width = 0.6,
            height = 0.35)


In [ ]:
### Figure S1: Group level exit thresholds by Experiment
threshold_by_effort_type_subj_experiment_2 <- choiceData_experiment_2 %>%
  filter(decision == "exit") %>%
  group_by(effort_level, travel_task, subject_id) %>%
  summarise(Threshold = mean(expectedReward, na.rm = TRUE),
            Threshold_sd = sd(expectedReward, na.rm = TRUE))

threshold_by_effort_type_experiment_2 <- threshold_by_effort_type_subj_experiment_2 %>%
  summarise(threshold = mean(Threshold, na.rm = TRUE),
            nsubj = n(),
            threshold_sem = sd(Threshold, na.rm = TRUE)/sqrt(nsubj)) %>%
  mutate(threshold_sem = ifelse(is.na(threshold_sem), 0, threshold_sem),
         block_str = case_when(effort_level == "low" & travel_task == "nback" ~
                                 "1-Back",
                               effort_level == "high" & travel_task == "nback" ~
                                 "3-Back",
                               effort_level == "low" & travel_task == "bp" ~
                                 "Smaller presses",
                               effort_level == "high" & travel_task == "bp" ~
                                 "Larger presses"),
         block_str = factor(block_str,
                            levels = c("1-Back",
                                       "3-Back",
                                       "Smaller presses",
                                       "Larger presses")),
         experiment = "Experiment 2")

threshold_by_effort_type_subj_experiment_1 <- choiceData_experiment_1 %>%
  filter(decision == "exit") %>%
  group_by(effort_level, travel_task, subject_id) %>%
  summarise(Threshold = mean(expectedReward, na.rm = TRUE),
            Threshold_sd = sd(expectedReward, na.rm = TRUE))

threshold_by_effort_type_experiment_1 <- threshold_by_effort_type_subj_experiment_1 %>%
  summarise(threshold = mean(Threshold, na.rm = TRUE),
            nsubj = n(),
            threshold_sem = sd(Threshold, na.rm = TRUE)/sqrt(nsubj)) %>%
  mutate(threshold_sem = ifelse(is.na(threshold_sem), 0, threshold_sem),
         block_str = case_when(effort_level == "low" & travel_task == "msit" ~
                                 "MSIT Congruent",
                               effort_level == "high" & travel_task == "msit" ~
                                 "MSIT Interference",
                               effort_level == "low" & travel_task == "bp" ~
                                 "Smaller presses",
                               effort_level == "high" & travel_task == "bp" ~
                                 "Larger presses"),
         block_str = factor(block_str,
                            levels = c("MSIT Congruent",
                                       "MSIT Interference",
                                       "Smaller presses",
                                       "Larger presses")),
         experiment = "Experiment 1")

threshold_max <- max(max(threshold_by_effort_type_experiment_2$threshold +
                         threshold_by_effort_type_experiment_2$threshold_sem),
                     max(threshold_by_effort_type_experiment_1$threshold +
                         threshold_by_effort_type_experiment_1$threshold_sem)) + 0.41
   # for plot to show best threshold

p_threshold_experiment_2 <- ggplot(data = threshold_by_effort_type_experiment_2) +
  geom_bar(aes(x = block_str,
               y = threshold,
               fill = effort_level),
           stat = "identity",
           position = position_dodge(),
           width = 0.6,
           size = 2) +
  geom_hline(aes(yintercept = 4.02),
             linetype = "dashed") +
  scale_fill_manual(values=c("grey30",
                             "grey50")) +
      geom_errorbar(aes(x = block_str,
                      ymax = threshold + threshold_sem,
                      ymin = threshold - threshold_sem),
                      width = 0,
                      size = 1,
                      position = position_dodge(width=0.9),
                      color = "black") +
  labs(title = "Experiment 2 (N-Back)",
       y = "Mean exit threshold (apples)",
       fill = "Effort level",
       tags = "B") +
  theme(text = element_text(size = 14,
                          family = "Times New Roman"),
        axis.text = element_text(size = 14),
        axis.text.x = element_text(angle = 45,
                                   vjust = 1,
                                   hjust = 1.1),
        axis.title.x = element_blank(),
        axis.title.y = element_blank(),
        legend.position = "none",
        panel.background = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.major.y = element_line(size = 0.5,
                                        linetype = 'solid',
                                        colour = "grey60")) +
  scale_y_continuous(limits = c(0, threshold_max),
                     breaks = scales::breaks_pretty())
p_threshold_experiment_1 <- ggplot(data = threshold_by_effort_type_experiment_1) +
  geom_bar(aes(x = block_str,
               y = threshold,
               fill = effort_level),
           stat = "identity",
           position = position_dodge(),
           width = 0.6,
           size = 2) +
  geom_hline(aes(yintercept = 6.78),
             linetype = "dashed") +
  scale_fill_manual(values=c("grey30",
                             "grey50")) +
      geom_errorbar(aes(x = block_str,
                      ymax = threshold + threshold_sem,
                      ymin = threshold - threshold_sem),
                      width = 0,
                      size = 1,
                      position = position_dodge(width=0.1),
                      color = "black") +
  labs(title = "Experiment 1 (MSIT)",
       y = "Group mean exit threshold (apples)",
       fill = "Effort level",
       tags = "A") +
  theme(text=element_text(size = 14,
                          family = "Times New Roman"),
        axis.text.y = element_text(size = 14),
        axis.text.x = element_text(size = 12,
                                   angle = 45,
                                   vjust = 1,
                                   hjust = 1.1),
        axis.title.x = element_blank(),
        legend.position = "none",
        panel.background = element_blank(),
        panel.grid.major.x = element_blank(),
        panel.grid.major.y = element_line(size = 0.5,
                                        linetype = 'solid',
                                        colour = "grey60")) +
  scale_y_continuous(limits = c(0, threshold_max),
                     breaks = scales::breaks_pretty())
figure_S1 <- ggdraw() +
  draw_plot(p_threshold_experiment_1,
            x = 0,
            y = 0,
            width = 0.5,
            height = 1) +
  draw_plot(p_threshold_experiment_2,
            x = 0.5,
            y = 0,
            width = 0.5,
            height = 1)
print(figure_S1)

In [ ]:
### Table S1: Group level posterior distributions.

cost_travel_stan_fit_group_experiment_1 <- cost_travel_stan_fit_experiment_1 %>%
    select(contains("beta_ms[")) %>%
    pivot_longer(everything(),
                 names_to = "params",
                 values_to = "params_val") %>%
    mutate(param_name = case_when(params == "beta_ms[1]" ~ cost_travel_fit_vars_titles[1],
                                  params == "beta_ms[2]" ~ cost_travel_fit_vars_titles[2],
                                  params == "beta_ms[3]" ~ cost_travel_fit_vars_titles[3],
                                  params == "beta_ms[4]" ~ cost_travel_fit_vars_titles[4],
                                  params == "beta_ms[5]" ~ cost_travel_fit_vars_titles[5]))
table_S1 <- cost_travel_stan_fit_group_experiment_1 %>%
  ungroup() %>%
  group_by(param_name) %>%
  summarise(mean = mean(params_val),
            lower = HDInterval::hdi(params_val, credMass = 0.95)[1],
            upper = HDInterval::hdi(params_val, credMass = 0.95)[2]) %T>%
  print()


Hierarchal bayesian model

In [ ]:
# ```{r load_stan_data}
setwd("~/hierarchal_bayesian_mvt_model")
data <- read_csv("foraging_mvt_stan_data_experiment_1.csv")
foraging_experiment_1_standata <- list(NS = unique(data$NS),
                               K = dim(data)[1],
                               choose_stay = data$choose_stay,
                               which_S = data$which_S,
                               total_reward = data$total_reward,
                               total_harvest_periods = data$total_harvest_periods,
                               number_trees = data$number_trees,
                               expected_reward = data$expected_reward,
                               is_Matching = data$is_Matching,
                               is_Mismatching = data$is_Mismatching,
                               is_small = data$is_small,
                               is_large = data$is_large)

# ```{r run_cmdstanr}
file <- file.path("~/hierarchal_bayesian_mvt_model/", "fit_mvt_stan_model_experiment_1.stan")
mod <- cmdstan_model(file)
mod$print()
# note the sampling can take hours
fit_experiment_1 <- mod$sample(
  data = foraging_experiment_1_standata,
  chains = 4,
  parallel_chains = 4,
  refresh = 150,
  iter_warmup = 500,
  iter_sampling = 1000,
  thin = 1
)


# ```{r save_stan_fit}
fit_experiment_1$summary()
cost_travel_experiment_1_fit_full <- fit_experiment_1$draws(format = "df")
setwd('~/hierarchal_bayesian_mvt_model/')
# save the draws and also the model fit object
# write_csv(cost_travel_experiment_1_fit_full, "cost_travel_experiment_1_fit_full.csv")
# saveRDS(fit_experiment_1, "fit_experiment_1.rds")
# load without rerunning the model
# cost_travel_experiment_1_fit_full <- read_csv("cost_travel_experiment_1_fit_full.csv")
# fit_experiment_1 <- readRDS("fit_experiment_1.rds")
